In [34]:
import numpy as np
import time
from copy import deepcopy
from scipy.linalg import block_diag

from Utils_Dipole.bsplines import Bspline
from Utils_Dipole.borisPushRelativistic import borisPushRelativistic
from Utils_Dipole.fieldInterpolation_V1 import fieldInterpolation
from Utils_Dipole.hotCurrentRelativistic import hotCurrentRelativistic
from Utils_Dipole.initialConditions import IC
from Utils_Dipole.dispersionSolver import solveDispersion
from Utils_Dipole.createBasis import createBasis
from Utils_Dipole.matrixAssembly import matrixAssembly



restart = 0                        # ... start the simulation from the beginning (0) or continue (1)                                
title = 'Results/simulation_data_T=100_Relativistic.txt' # ... directory to save results


# ... physical parameters
eps0 = 1.0                         # ... vacuum permittivity
mu0 = 1.0                          # ... vacuum permeability
c = 1.0                            # ... speed of light
qe = -1.0                          # ... electron charge
me = 1.0                           # ... electron mass
B0z = 1.0                          # ... background magnetic field in z-direction at z = Lz/2
wce = qe*B0z/me                    # ... electron cyclotron frequency
wpe = 2*np.abs(wce)                # ... cold electron plasma frequency
nuh = 6e-2                         # ... ratio of cold/hot electron densities (nh/nc)
nh = nuh*wpe**2                    # ... hot electron density
wpar = 0.2*c                       # ... parallel thermal velocity of energetic particles
wperp = 0.63*c                     # ... perpendicular thermal velocity of energetic particles
xi = 8.62e-5*0                     # ... inhomogeneity factor of background magnetic field
# ...




# ... parameters for initial conditions
k = 2                              # ... wavenumber of initial wave fields
ini = 3                            # ... initial conditions for wave fields
amp = 1e-6                         # ... amplitude of initial wave fields
eps = 0.0                          # ... amplitude of spatial pertubation of distribution function 
# ...



# ... numerical parameters
Lz = 2*np.pi/k                     # ... total length of z-domain
Nz = 200                           # ... number of elements z-direction
Nd = 16                            # ... number of damping elements at each end
T = 60.0                           # ... simulation time
dt = 0.05                          # ... time step
p = 3                              # ... degree of B-spline basis
Lv = 8                             # ... length of v-domain in each direction (vx,vy,vz)
Nv = 76                            # ... number of cells in each v-direction (vx,vy,vz)
Np = np.int(5e4)                   # ... number of energetic simulation particles 
# ...



# ... create parameter list
pa = np.zeros(8*Nz + 1)

pa[0]  = eps0
pa[1]  = mu0
pa[2]  = c
pa[3]  = qe 
pa[4]  = me 
pa[5]  = B0z 
pa[6]  = wce 
pa[7]  = wpe 
pa[8]  = nuh 
pa[9]  = nh 
pa[10] = wpar 
pa[11] = wperp 
pa[12] = k 
pa[13] = ini 
pa[14] = amp 
pa[15] = eps 
pa[16] = Lz 
pa[17] = Nz 
pa[18] = T 
pa[19] = dt 
pa[20] = p 
pa[21] = Lv 
pa[22] = Nv 
pa[23] = Np 
pa[24] = xi
# ...



# ... discretization parameters
dz = Lz/Nz
zj = np.linspace(0,Lz,Nz+1)

Nt = np.int(T/dt)
tn = np.linspace(0,T,Nt+1)

dv = Lv/Nv
vj = np.linspace(-Lv/2,Lv/2,Nv+1)
# ...






# ... system matrices for fluid electrons and electromagnetic fields and define background magnetic field B0
A10 = np.array([0,0,0,c**2,0,0])
A11 = np.array([0,0,-c**2,0,0,0])
A12 = np.array([0,-1,0,0,0,0])
A13 = np.array([1,0,0,0,0,0])
A14 = np.array([0,0,0,0,0,0])
A15 = np.array([0,0,0,0,0,0])
A1 = np.array([A10,A11,A12,A13,A14,A15])

A20 = np.array([0,0,0,0,mu0*c**2,0])
A21 = np.array([0,0,0,0,0,mu0*c**2])
A22 = np.array([0,0,0,0,0,0])
A23 = np.array([0,0,0,0,0,0])
A24 = np.array([-eps0*wpe**2,0,0,0,0,-wce])
A25 = np.array([0,-eps0*wpe**2,0,0,wce,0])
A2 = np.array([A20,A21,A22,A23,A24,A25])

s = int(np.sqrt(A1.size))

def B0(x,y,z,Lz,B0z,xi):
    
    Bx = -x*(z - Lz/2)*B0z*xi
    By = -y*(z - Lz/2)*B0z*xi
    Bz = B0z*(1 + xi*(z - Lz/2)**2)
    
    return np.transpose(np.array([Bx,By,Bz]))
# ...




# ... time integration 
def update(uj,particles,Ep,Bp,dt):
    
    
    # ... save old positions
    zold = deepcopy(particles[:,0])
    # ...
    
    
    # ... update particle velocities from n-1/2 to n+1/2 with fields at time n and positions from n to n+1 with velocities at n+1/2
    znew,unew = borisPushRelativistic(particles,dt,Bp,Ep,qe,me,Lz,c)
    # ...
    
    
    # ... update weights with control variate
    wnew = w0 - Maxwell(unew[:,0],unew[:,1],unew[:,2])/g0
    # ...
    
    
    # ... compute hot electron current densities
    jhnew = hotCurrentRelativistic(unew,1/2*(znew + zold),wnew,zj,bsp,p,qe,c)
    # ...
     
    
    # ... assemble right-hand side of weak formulation
    for i in range(0,Nb):
        il = s*i
        Fh[il+0] = -c**2*mu0*jhnew[2*i]
        Fh[il+1] = -c**2*mu0*jhnew[2*i+1]
    # ...
    
    
    # ... time integration of E,B,jc from n to n+1 with Crank-Nicolson method (use hot current density at n+1/2) 
    ujnew = np.dot(LHSinv,np.dot(RHS,uj) + dt*Fh)
    
    #ujnew[0:s*(Nd+1)] = fl*ujnew[0:s*(Nd+1)]
    #ujnew[s*(Nb-Nd-1):] = fr*ujnew[s*(Nb-Nd-1):]
    # ...
    
    
    # ... compute fields at particle positions with new fields (wave + background) 
    Epnew,Bpnew = fieldInterpolation(znew,zj,bsp,ujnew,p)
    
    Bpznew = B0(0,0,znew,Lz,B0z,xi)[:,2]
    rho = -me/qe*np.cross(unew,np.array([0,0,1]))/Bpznew[:,None]
    
    Bpnew += B0(rho[:,0],rho[:,1],znew,Lz,B0z,xi)[:,0:2]
    # ...
    
    return znew,unew,wnew,jhnew,ujnew,Epnew,Bpnew,Bpznew
# ...






if restart == 0:



    
    # ... initial energetic particle distribution function (perturbed anisotropic Maxwellian)
    def fh0(z,ux,uy,uz,eps,B0):
        
        xiB = 1 - np.linalg.norm(B0(0,0,0,Lz,B0z,xi))/np.linalg.norm(B0(0,0,z,Lz,B0z,xi),axis = 1)
        xiz = 1 + (wperp**2/wpar**2 - 1)*xiB
        
        return (1 + eps*np.cos(k*z))*nh/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-uz**2/(2*wpar**2) - xiz*(ux**2 + uy**2)/(2*wperp**2))
    # ...


    # ... Maxwellian for control variate
    def Maxwell(ux,uy,uz):
        return nh/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-uz**2/(2*wpar**2) - (ux**2 + uy**2)/(2*wperp**2))
    # ...


    # ... sampling distribution for initial markers
    def g_sampling(ux,uy,uz):
        return 1/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-uz**2/(2*wpar**2) - (ux**2 + uy**2)/(2*wperp**2))*1/Lz
    # ...

    
    
    # ... masking function to damp wave fields near boundaries
    def fDL(n,Nd,N):
        
        a = -1/Nd**2
        b = -2*a*Nd
        
        return a*n**2 + b*n

    def fDR(n,Nd,N):
    
        a = 1/(2*N*(N - Nd) - (N - Nd)**2 - N**2)
        b = -2*a*(N-Nd)
        c = 2*a*N*(N - Nd) - a*N**2

        return a*n**2 + b*n + c
    
    nl = np.linspace(0,Nd,Nd+1)
    nr = np.linspace(Nz-Nd,Nz,Nd+1)
    
    fl = np.repeat(fDL(nl,Nd,Nz),s)
    fr = np.repeat(fDR(nr,Nd,Nz),s)
    # ...
    


    # ... create periodic B-spline basis and quadrature grid
    bsp,N,quad_points,weights = createBasis(Lz,Nz,p)
    # ...



    # ... matrices for linear system
    Nb = N - p                        # ... number of unique B-splines for periodic boundary conditions

    uj = np.zeros(s*Nb)               # ... coefficients for Galerkin approximation
    Fh = np.zeros(s*Nb)               # ... RHS of matrix system

    Mblock = np.zeros((s*Nb,s*Nb))    # ... block mass matrix    
    Cblock = np.zeros((s*Nb,s*Nb))    # ... block convection matrix
    Dblock = np.zeros((s*Nb,s*Nb))    # ... block background field matrix

    u0 = np.zeros((Nb,s))             # ... L2-projection of initial conditions

    A1block = block_diag(*([A1]*Nb))  # ... block system matrix A1
    A2block = block_diag(*([A2]*Nb))  # ... block system matrix A2
    # ...





    # ... assemble mass, convection and background field matrices
    timea = time.time()

    M,C,D = matrixAssembly(bsp,p,Nz,weights,quad_points,B0z,xi,Lz)

    timeb = time.time()
    print('time for matrix assembly: ' + str(timeb - timea))
    # ...





    # ... assemble u0
    timea = time.time()

    for qu in range (0,s):
        for ie in range(0,Nz):
            for il in range(0,p+1):

                i = il + ie

                value_u0 = 0.0

                for g in range(0,p+1):
                    gl = ie*(p+1) + g

                    value_u0 += weights[gl]*IC(quad_points[gl],ini,amp,k,omega = 0)[qu]*bsp(quad_points[gl],i,0)

                u0[i%Nb,qu] += value_u0

    timeb = time.time()
    print('time for vector assembly: ' + str(timeb - timea))
    # ...





    # ... L2-projection of initial conditions + damping
    uini = np.dot(np.linalg.inv(M),u0)
    uj = np.reshape(uini,s*Nb)
    
    #uj[0:s*(Nd+1)] = fl*uj[0:s*(Nd+1)]
    #uj[s*(Nb-Nd-1):] = fr*uj[s*(Nb-Nd-1):]
    # ...

    


    # ... construct block mass, convection and field matrices
    for i in range(0,Nb):
        for j in range(0,Nb):
            l = s*i
            m = s*j
            Mblock[l:l+s,m:m+s] = np.identity(s)*M[i,j]
            Cblock[l:l+s,m:m+s] = np.identity(s)*C[i,j]
            
            A20loc = np.array([0,0,0,0,mu0*c**2*M[i,j],0])
            A21loc = np.array([0,0,0,0,0,mu0*c**2*M[i,j]])
            A22loc = np.array([0,0,0,0,0,0])
            A23loc = np.array([0,0,0,0,0,0])
            A24loc = np.array([-eps0*wpe**2*M[i,j],0,0,0,0,-qe/me*D[i,j]])
            A25loc = np.array([0,-eps0*wpe**2*M[i,j],0,0,qe/me*D[i,j],0])
            A2loc = np.array([A20loc,A21loc,A22loc,A23loc,A24loc,A25loc])
            Dblock[l:l+s,m:m+s] = A2loc
    # ...





    # ... create particles (z,vx,vy,vz,wk) and sample positions and velocities according to sampling distribution
    particles = np.zeros((Np,5))
    particles[:,0] = np.random.rand(Np)*Lz
    particles[:,1] = np.random.randn(Np)*wperp
    particles[:,2] = np.random.randn(Np)*wperp
    particles[:,3] = np.random.randn(Np)*wpar
    # ...




    # ... parameters for control variate
    g0 = g_sampling(particles[:,1],particles[:,2],particles[:,3])
    w0 = fh0(particles[:,0],particles[:,1],particles[:,2],particles[:,3],eps,B0)/g_sampling(particles[:,1],particles[:,2],particles[:,3])
    # ...






    # ... initial fields at particle positions
    Ep = np.zeros((Np,3))
    Bp = np.zeros((Np,3))
    
    timea = time.time()
    
    Ep[:,0:2],Bp[:,0:2] = fieldInterpolation(particles[:,0],zj,bsp,uj,p)
    
    
    Bp[:,2] = B0(0,0,particles[:,0],Lz,B0z,xi)[:,2]
    rho = -me/qe*np.cross(particles[:,1:4],np.array([0,0,1]))/Bp[:,2][:,None]
    
    Bp[:,0:2] += B0(rho[:,0],rho[:,1],particles[:,0],Lz,B0z,xi)[:,0:2]

    timeb = time.time()
    print('time for intial field interpolation: ' + str(timeb - timea))
    # ...





    # ... initialize velocities by pushing back by -dt/2 and compute weights
    timea = time.time()

    particles[:,1:4] = borisPushRelativistic(particles,-dt/2,Bp,Ep,qe,me,Lz,c)[1]
    particles[:,4] = w0 - Maxwell(particles[:,1],particles[:,2],particles[:,3])/g0

    timeb = time.time()
    print('time for intial particle push: ' + str(timeb - timea))
    #





    # ... compute matrices for field update
    timea = time.time()

    LHS = Mblock + 1/2*dt*np.dot(Cblock,A1block) + 1/2*dt*np.dot(Mblock,A2block)
    RHS = Mblock - 1/2*dt*np.dot(Cblock,A1block) - 1/2*dt*np.dot(Mblock,A2block)
    LHSinv = np.linalg.inv(LHS)

    timeb = time.time()
    print('time for update matrix computation: ' + str(timeb - timea))
    # ...





    # ... create data file and save parameters (first row) and initial fields (second row)
    file = open(title,'ab')


    np.savetxt(file,np.reshape(pa,(1,8*Nb + 1)),fmt = '%1.6e')

    data = np.append(uj,np.zeros(2*Nb))
    data = np.append(data,tn[0])
    np.savetxt(file,np.reshape(data,(1,8*Nb + 1)),fmt = '%1.6e')
    # ...



    # ... time loop
    print('total number of time steps: ' + str(Nt))

    for n in range(0,Nt):

        if n%50 == 0:
            print('time steps finished: ' + str(n))

        particles[:,0],particles[:,1:4],particles[:,4],jh,uj,Ep[:,0:2],Bp[:,0:2],Bp[:,2] = update(uj,particles,Ep,Bp,dt)

        # ... add data to file
        data = np.append(uj,jh)
        data = np.append(data,tn[n+1])
        np.savetxt(file,np.reshape(data,(1,8*Nb + 1)),fmt = '%1.6e')
        # ...
    # ...


    file.close()
    

    
    
    
    
if restart == 1:
    
    # ... open data file that hasn't been finished yet
    file = open('Results/simulation_data_T=20(test).txt','ab')
    # ...


    # ... time loop
    print('total number of time steps: ' + str(Nt))

    for i in range(n,Nt):

        if i%50 == 0:
            print('time steps finished: ' + str(i))

        particles[:,0],particles[:,1:4],particles[:,4],jh,uj,Ep[:,0:2],Bp[:,0:2],Bp[:,2] = update(uj,particles,Ep,Bp,dt)

        # ... add data to file
        data = np.append(uj,jh)
        data = np.append(data,tn[i+1])
        np.savetxt(file,np.reshape(data,(1,8*Nb + 1)),fmt = '%1.6e')
        # ...
    # ...


    file.close() 

time for matrix assembly: 2.5712392330169678
time for vector assembly: 1.0704615116119385
time for intial field interpolation: 0.12770986557006836
time for intial particle push: 0.014714241027832031
time for update matrix computation: 1.4836106300354004
total number of time steps: 1200
time steps finished: 0
time steps finished: 50
time steps finished: 100
time steps finished: 150
time steps finished: 200
time steps finished: 250
time steps finished: 300
time steps finished: 350
time steps finished: 400
time steps finished: 450
time steps finished: 500
time steps finished: 550
time steps finished: 600
time steps finished: 650
time steps finished: 700
time steps finished: 750
time steps finished: 800
time steps finished: 850
time steps finished: 900
time steps finished: 950
time steps finished: 1000
time steps finished: 1050
time steps finished: 1100
time steps finished: 1150
